In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import fnmatch
import os
from matplotlib import colors
import matplotlib.pyplot as plt

2021-12-13 00:41:45.540444: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-13 00:41:45.540458: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from sklearn.model_selection import train_test_split

## Read Input Data

In [3]:
def load_data(path_to_file,maze_size):
    X = []
    y = []
    f = open(path_to_file,'rb')
    
    try:
        while True:
            x1 = np.load(f).reshape(maze_size,maze_size)
            y1 = np.load(f)
            
            if(len(y1)<4):
                print(y1)
                continue
            X.append(x1)
            y.append(y1)
    except:
            # print("eof")
            pass
    return X,y
    

In [4]:
def read_dataset(maze_size):
    final_X = []
    final_y = []
    dirs = [d for d in os.listdir("./") if fnmatch.fnmatch(d,'dataset*')]
    # print(dirs)
    # dirs = dirs[-1:-2:-1]
    print(dirs)
    for directory in dirs:
        for f in os.listdir("./"+directory+"/"):
            if fnmatch.fnmatch(f,'data*'):
                X,y = load_data("./"+directory+"/"+f,maze_size)
                # print(y.shape,type(y))
                if(len(X)==len(y)):
                # print(len(X))
                    final_X += X
                    final_y += y
                else:
                    print("wrong inputs in :","./"+directory+"/"+f)
    
    return np.array(final_X),final_y

In [339]:
X,y = read_dataset(4)

['dataset9', 'dataset3', 'dataset', 'dataset4', 'dataset0', 'dataset7', 'dataset5', 'dataset2', 'dataset8', 'dataset10', 'dataset6', 'dataset1']


In [340]:
y = np.asarray(y)

In [341]:
y.shape

(1623551, 4)

In [342]:
X.shape

(1623551, 4, 4)

In [343]:
## Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [344]:
# train_in = np.reshape(X_train, (-1,4,4))
# test_in = np.reshape(X_test, (-1,4,4))
# train_out = np.reshape(y_train, (-1,4,1))
# test_out = np.reshape(y_test, (-1,4,1))
# print(train_in.shape,test_in.shape, type(train_in))

## Display Data

In [5]:
def display_input_map(matrix):
    """
    This function is used to display maze with traversed path(highlighted)
    trajectory_path: path traversed by the agent so far
    matrix: maze of the problem
    """
    temp = matrix.copy()
    colormap = colors.ListedColormap(["green","black","white","blue"])
    plt.imshow(temp, cmap=colormap)
    plt.show()

In [6]:
def display_collected_data(X,y):
    for i in range(len(X)):
        display_input_map(np.asmatrix(X[i]))
        if y[i][0]==1:
            print("Up")
        elif y[i][1]==1:
            print("down")
        elif y[i][2]==1:
            print("left")
        elif y[i][3]==1:
            print("right")

In [347]:
sample_range = 100000
indices = list(np.random.choice(sample_range,10))
# print(indices)
sample_x = [X[index] for index in indices]
sample_y = [y[index] for index in indices]

In [348]:
# display_collected_data(sample_x,sample_y,4)

## Build Model

In [349]:
model = keras.models.Sequential([
    keras.Input(shape = (4,4)),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(4, activation= None),
    keras.layers.Softmax()
])

In [350]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 16)                0         
                                                                 
 dense_24 (Dense)            (None, 100)               1700      
                                                                 
 dense_25 (Dense)            (None, 50)                5050      
                                                                 
 dense_26 (Dense)            (None, 4)                 204       
                                                                 
 softmax_1 (Softmax)         (None, 4)                 0         
                                                                 
Total params: 6,954
Trainable params: 6,954
Non-trainable params: 0
_________________________________________________________________


In [351]:
model.compile(loss = tf.keras.losses.binary_crossentropy, optimizer = tf.optimizers.Adam(),metrics=['accuracy'])

In [352]:
# [print(i.shape, i.dtype) for i in model.inputs]
# [print(o.shape, o.dtype) for o in model.outputs]
# [print(l.name, l.input_shape, l.dtype) for l in model.layers]

In [353]:
history = model.fit(X_train, y_train, epochs=25, batch_size=50,verbose=1, validation_split=0.2)

Epoch 1/25
17405/17405 [==============================] - 51s 3ms/step - loss: 0.0194 - accuracy: 0.9859 - val_loss: 2.4102e-04 - val_accuracy: 1.0000
Epoch 2/25
17405/17405 [==============================] - 51s 3ms/step - loss: 4.7180e-04 - accuracy: 0.9997 - val_loss: 1.1601e-05 - val_accuracy: 1.0000
Epoch 3/25
17405/17405 [==============================] - 50s 3ms/step - loss: 5.7415e-04 - accuracy: 0.9997 - val_loss: 2.1055e-06 - val_accuracy: 1.0000
Epoch 4/25
17405/17405 [==============================] - 49s 3ms/step - loss: 8.0819e-04 - accuracy: 0.9997 - val_loss: 2.4631e-05 - val_accuracy: 1.0000
Epoch 5/25
17405/17405 [==============================] - 50s 3ms/step - loss: 4.0321e-04 - accuracy: 0.9998 - val_loss: 4.3672e-07 - val_accuracy: 1.0000
Epoch 6/25
17405/17405 [==============================] - 48s 3ms/step - loss: 1.9287e-04 - accuracy: 0.9999 - val_loss: 5.3467e-08 - val_accuracy: 1.0000
Epoch 7/25
17405/17405 [==============================] - 48s 3ms/step - l

In [357]:
model.save('./model_weights/model_1_v2.model')

INFO:tensorflow:Assets written to: ./model_weights/model_1_v2.model/assets


In [354]:
testing_model = model.evaluate(X_test,y_test,verbose=2)

16743/16743 - 23s - loss: 0.0000e+00 - accuracy: 1.0000 - 23s/epoch - 1ms/step


In [355]:
testing_acc = testing_model[1]*100
# Printing the accuracy
print('Test Accuracy - ', testing_acc,'%')

Test Accuracy -  100.0 %


In [356]:
testing_model

[0.0, 1.0]

In [ ]:
model.metrics_names

In [360]:
predictions = model.predict(X_test[0:1])

In [361]:
X[0:1]

array([[[4, 3, 3, 3],
        [3, 3, 3, 3],
        [3, 3, 3, 3],
        [3, 3, 3, 3]]])

In [332]:
print(y_test[:3],"\n",predictions)

[[0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]] 
 [[1.9317674e-06 9.9999583e-01 2.3033076e-06 2.2856215e-13]
 [2.8126976e-34 3.0568835e-06 3.2739661e-10 9.9999690e-01]
 [5.4714701e-31 2.7400508e-06 1.2310340e-11 9.9999726e-01]]


In [ ]:
tf.keras.losses.binary_crossentropy

In [ ]:
import matplotlib.pyplot as plt
plot_x = history.history['loss']
plot_y = list(np.arange(len(x)))

In [7]:
model_test = keras.models.load_model('model_weights/model_1_accuracy_100.model')

2021-12-13 00:43:09.080194: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-13 00:43:09.080238: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (coco): /proc/driver/nvidia/version does not exist
2021-12-13 00:43:09.080552: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
%run data_collection.ipynb

In [11]:
initialize(4,0.3)

In [12]:
final_grid = encode_grid(grid,[])
final_grid[0,0] = 4

In [13]:
np.asarray([final_grid.tolist()])

array([[[4, 3, 3, 3],
        [3, 3, 3, 2],
        [2, 3, 3, 3],
        [3, 3, 2, 3]]])

In [14]:
model_test.predict(np.asarray([final_grid.tolist()]))

array([[0.0000000e+00, 9.9634355e-01, 2.0873466e-12, 3.6564714e-03]],
      dtype=float32)